# Imports

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import json

# Load data

In [ ]:
test = []
with open(f'/content/gdrive/MyDrive/Independent study - Max & Carlos/data/test.jsonl') as f_pdf:
    for line in f_pdf:
        dicti = json.loads(line)
        test.append(dicti)
test_res = []
with open(f'/content/gdrive/MyDrive/Independent study - Max & Carlos/data/zeroshot_test_res_on_test.jsonl') as f_pdf:
    for line in f_pdf:
        dicti = json.loads(line)
        test_res.append(dicti)

In [ ]:
print(len(test), len(test_res))

# Evaluate

In [ ]:
## Multivers Match Rate
suc = 0
total = 0
for i in range(len(test)):
    total += len(test[i]['doc_ids'])
    for key in test[i]['doc_ids']:
      key = str(key)
      if key in test_res[i]['evidence'] and test_res[i]['evidence'][key]['label'] == test[i]['evidence'][key][0]['label']:
        suc += 1
      elif key not in test_res[i]['evidence'] and key not in test[i]['evidence']:
        suc += 1
print(suc, total)
print(suc / total)

79 107
0.7383177570093458
{'claim': 'It was shown that cSMAC formation could not enhance signaling by weak ligands despite using manipulations that enhance cSMAC formation (Cemerski et al. 2007).', 'id': 1058, 'evidence': {'306006': [{'sentences': [0, 1, 2, 3, 4, 5], 'label': 'CONTRADICT'}]}, 'citance_id': 8316253, 'doc_ids': [306006]}
{'id': 1058, 'evidence': {'306006': {'label': 'SUPPORT', 'sentences': [4]}}}


In [ ]:
c_n = 0 # num of contradict
c_tp = 0 # num correct
c_fp = 0
s_n = 0
s_tp = 0
s_fp = 0
n_n = 0
n_tp = 0
n_fp = 0
#### fn = n - tp

for i in range(len(test)):
  t = test[i] # actual
  t_res = test_res[i] # predictions
  if t['id'] != t_res['id']:
    print("something is amuck hjere")
  for doc_id in t['doc_ids']:
    label = "" # label is the actual value
    doc_id = str(doc_id)
    if doc_id in t['evidence']: # finding value
      if t['evidence'][doc_id][0]['label'] == "CONTRADICT":
        c_n += 1
      else:
        s_n += 1
      label = t['evidence'][doc_id][0]['label']
    else:
      n_n += 1
    if doc_id not in t_res['evidence'] and not label: # NEI TP AND FP
      n_tp += 1
    elif doc_id not in t_res['evidence'] and label:
      n_fp += 1
    else: # it made a prediction (not nei)
      if label == t_res['evidence'][doc_id]['label']: ## TP FOR S AND C
        if label == "CONTRADICT":
          c_tp += 1
        else:
          s_tp += 1
      else:
        if label == "CONTRADICT": ## FP FOR S AND C
          s_fp += 1
        else:
          c_fp += 1

c_fn = c_n - c_tp
s_fn = s_n - s_tp
n_fn = n_n - n_tp

In [ ]:
print(c_n)
print(s_n)
print(n_n)

# Calculate precision and recall for Class C
c_precision = c_tp / (c_tp + c_fp)
c_recall = c_tp / (c_tp + c_fn)

# Calculate precision and recall for Class S
s_precision = s_tp / (s_tp + s_fp)
s_recall = s_tp / (s_tp + s_fn)

# Calculate precision and recall for Class N
n_precision = n_tp / (n_tp + n_fp)
n_recall = n_tp / (n_tp + n_fn)

print("Precision for Class Contradict:", c_precision)
print("Recall for Class Contradict:", c_recall)

print("Precision for Class Supports:", s_precision)
print("Recall for Class Supports:", s_recall)

print("Precision for Class NEI:", n_precision)
print("Recall for Class NEI:", n_recall)


44
32
31
Precision for Class Contradict: 1.0
Recall for Class Contradict: 0.6136363636363636
Precision for Class Supports: 0.75
Recall for Class Supports: 0.65625
Precision for Class NEI: 0.5961538461538461
Recall for Class NEI: 1.0


## Contradicts: did not guess contradict enough, but was accurate when guessing
## Support: guessed decently
## NEI: guessed NEI too much, all NEIs were found, but too many non-NEIs were labeled as NEI